In [ ]:
import geopandas as gpd
import ipywidgets
import pandas as pd
import pydeck as pdk

## Load in our data sets
df = pd.read_csv('https://raw.githubusercontent.com/ajduberstein/sf_growth/master/public/data/business.csv')

# Handle null values which may not serialize into deck.gl
df = df.fillna(0)

# Load in neighborhood boundaries for San Francisco
gdf = gpd.read_file('https://raw.githubusercontent.com/ajduberstein/sf_growth/master/public/data/neighborhoods.geojson')

# Zoom to the middle 80% of the business data (presumably centered in San Francisco)
viewport = pdk.data_utils.compute_view(df[['lng', 'lat']], 0.8)


# Create two scatter plots, one which will turn orange when new businesses are added
# and one which cumulatively tracks all businesses opened until that point

PURPLE = [154, 18, 179, 200]
ORANGE = [255, 98, 0]

def make_scatterplot(data, fill_color):
    return pdk.Layer(
        "ScatterplotLayer",
        data=data,
        pickable=True,
        get_position=['lng', 'lat'],
        radius_min_pixels=1,
        # Radius of the point in meters
        get_radius=10,
        radius_width_min_pixels=2,
        get_fill_color=fill_color,
    )

base_layer = make_scatterplot(df, PURPLE)
new_layer = make_scatterplot([], ORANGE)

# Define the GeoJsonLayer for the neighborhood boundaries

neighborhood_boundaries = pdk.Layer(
    "GeoJsonLayer",
    data=gdf,
    stroked=True,
    filled=False,
    line_width_min_pixels=2,
    get_radius=1
)

# Merge the visualization components
layers = [new_layer, base_layer, neighborhood_boundaries]
deck = pdk.Deck(
    layers,
    initial_view_state=viewport,
    map_provider=None,
    tooltip={'text': '{business_name}, opened {start_date}'}
)

# Render
display(deck.show())
year = ipywidgets.HTML()
display(year)

# Play the data forward in time, from 1950-2018 (takes 68 seconds)
def play():
    global deck
    global df
    global year
    for i in range(int(df.start_date.min()), int(df.start_date.max()) + 1):
        deck.layers[0].data = df[df['start_date'] == i]
        deck.layers[1].data = df[df['start_date'] < i]
        year.value = f'<b>{i}</b>'
        deck.update()
        time.sleep(1)
        

In [ ]:
play()